In [ ]:
# @title
!pip install -r https://raw.githubusercontent.com/LizaAmatya/FakeNewsDetection/main/requirements.txt?token=GHSAT0AAAAAACJI6OXTVIRQEN524EL4R3GIZLDRYUA

In [ ]:
# @title
import os
import pandas as pd
import tensorflow as tf
import tensorflow_text
from transformers import BertTokenizer, TFBertForSequenceClassification, DistilBertTokenizer, TFDistilBertForSequenceClassification
from tensorflow.keras.utils import to_categorical
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

In [ ]:
# BASE_DIR = os.path.dirname(os.path.abspath(__file__))
BASE_DIR = "/content/sample_data"

In [ ]:
os.environ["TF_AUTOTUNE"] = "1"
tf.keras.backend.set_floatx('float16')

In [ ]:
# # Enable autotuning
# tf.config.threading.set_intra_op_parallelism_threads(1)
# tf.config.threading.set_inter_op_parallelism_threads(1)
# tf.config.threading.set_intra_op_parallelism_threads(1)
# tf.config.threading.set_inter_op_parallelism_threads(1)

In [ ]:
train_path = os.path.join(BASE_DIR, 'dataset/train.tsv')
test_path = os.path.join(BASE_DIR, 'dataset/test.tsv')
validation_path = os.path.join(BASE_DIR, 'dataset/validation.tsv')

In [ ]:
column_labels = ['row', 'json_ids', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state', 'affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'lies_counts', 'context', 'justification']

In [ ]:
# Data Frames
train = pd.read_csv(train_path, sep="\t", header=None, names=column_labels)
test = pd.read_csv(test_path, sep="\t", header=None, names=column_labels)
valid = pd.read_csv(validation_path, sep="\t", header=None, names=column_labels)

In [ ]:
# Fill nan (empty boxes) with 0
train = train.fillna('None')
test = test.fillna('None')
val = valid.fillna('None')
print(train['label'])
print(val['label'])

0          false
1      half-true
2    mostly-true
3          false
4      half-true
5           true
6    barely-true
7      half-true
8      half-true
9    mostly-true
Name: label, dtype: object
0       barely-true
1        pants-fire
2             false
3         half-true
4         half-true
           ...     
1279      half-true
1280    mostly-true
1281           true
1282          false
1283    barely-true
Name: label, Length: 1284, dtype: object


In [ ]:
labels = train['label']
label_mapping = {label: idx for idx, label in enumerate(labels.unique())}
label_mapping.update({'pants-fire':5})

num_of_classes=len(label_mapping)
train = train[:100]
val = val[:50]
# print(num_of_classes)

{'false': 0, 'half-true': 1, 'mostly-true': 2, 'true': 3, 'barely-true': 4, 'pants-fire': 5}


In [94]:
train['label_encoded'] = train['label'].map(label_mapping)
print(train['label_encoded'])

0    0
1    1
2    2
3    0
4    1
5    3
6    4
7    1
8    1
9    2
Name: label_encoded, dtype: int64


In [95]:
val['label_encoded'] = val['label'].map(label_mapping)
print(val['label_encoded'])

0    4
1    5
2    0
3    1
4    1
5    0
6    3
7    0
8    3
9    1
Name: label_encoded, dtype: int64


In [ ]:
# train_one_hot_labels = to_categorical(train['label_encoded'], num_classes=num_of_classes)
# val_one_hot_labels = to_categorical(val['label_encoded'], num_classes=num_of_classes)

In [96]:
# model_name = 'experts_wiki_books'
# model_name = 'bert-base-uncased'
model_name = 'distilbert-base-uncased'
# tokenizer = BertTokenizer.from_pretrained(model_name, max_length=128)
tokenizer = DistilBertTokenizer.from_pretrained(model_name, max_length=128)
vocab_size = 10000
embedding_dim = 32

In [ ]:
# Create a custom embedding layer
# custom_embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)

In [ ]:
# Only using statement data at first
# Tokenize the statement data
train_encoded_statement_data = tokenizer(
    train['statement'].to_list(),
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors='tf'
)

print(train_encoded_statement_data)

train_labels = train['label'].tolist()

In [98]:
val_encoded_statement_data = tokenizer(
    val['statement'].tolist(),
    padding=True,
    truncation=True,
    return_tensors='tf'
)

val_labels = val['label'].tolist()

In [99]:
# Create TensorFlow dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': train_encoded_statement_data['input_ids'],
        'attention_mask': train_encoded_statement_data['attention_mask']
    },
    train['label_encoded'] ))  # using one-hot encoded labels when CategoricalCrossEntropy used,
                            # and when using SparseCrossEntropy use train['label_encoded'] which is int rep for labels : 0, 1, 2 ..5

In [100]:
val_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': val_encoded_statement_data['input_ids'],
        'attention_mask': val_encoded_statement_data['attention_mask']
    },
    val['label_encoded'] ))  # using one-hot encoded labels when CategoricalCrossEntropy used,
                            # and when using SparseCrossEntropy use train['label_encoded'] which is int rep for labels : 0, 1, 2 ..5

In [101]:
# Limiting the dataset
limit = 10
limited_train_dataset = train_dataset.take(limit)

In [102]:
# model = TFBertForSequenceClassification.from_pretrained(model_name)
model = TFDistilBertForSequenceClassification.from_pretrained(model_name)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [103]:
num_epochs = 1
batch_size = 32

In [104]:
# # Create a new model with the BERT base and the custom output layer

input_ids = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='attention_mask')

Custom embedding layer

custom_embeddings = custom_embedding_layer(input_ids)

In [106]:
# Adding a dense layer for the output
dense_layer = tf.keras.layers.Dense(num_of_classes, activation='softmax', name='dense_output')
bert_output = model([input_ids, attention_mask])
cls_token = bert_output.logits
dense_output = dense_layer(cls_token)
# dense_output_expanded = tf.keras.layers.Reshape((1, 6))(dense_output)

In [107]:
# combined_embeddings = tf.keras.layers.Concatenate(axis=-1)([dense_output_expanded, custom_embeddings])
output = tf.keras.layers.Dense(num_of_classes, activation='softmax')(dense_output)

In [108]:
# Create the final model
custom_model = tf.keras.Model(inputs=model.input, outputs=output)

In [109]:
custom_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(),        #BinaryCrossEntropy for binary classification; for now lets only classify acc to data: 6 classes
            metrics=tf.keras.metrics.SparseCategoricalAccuracy())  # or use ['accuracy']

In [110]:
custom_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, None)]               0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 tf_distil_bert_for_sequenc  TFSequenceClassifierOutput   6695501   ['input_ids[0][0]',           
 e_classification_1 (TFDist  (loss=None, logits=(None,    0          'attention_mask[0][0]']      
 ilBertForSequenceClassific  2),                                                            

In [111]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(BASE_DIR, 'model_checkpoint'),  # Specify the path to save the checkpoint
    save_best_only=True,  # Save only the best model based on the validation loss
    monitor='val_loss',  # Monitor the validation loss
    mode='min',  # Mode can be 'min' or 'max' depending on the monitored metric
    verbose=1  # Show progress while saving
)

In [112]:
from tqdm import tqdm
from tensorflow.keras.callbacks import Callback

train_steps_per_epoch = len(limited_train_dataset)
tqdm_callback = tf.keras.callbacks.LambdaCallback(
    on_epoch_begin=lambda epoch, logs: tqdm(total=train_steps_per_epoch, position=0, desc="Epoch", unit="batch"),
    on_epoch_end=lambda epoch, logs: tqdm.write(f'Epoch {epoch + 1}/{num_epochs}, Loss: {logs["loss"]}, Accuracy: {logs["sparse_categorical_accuracy"]}, Val Loss: {logs["val_loss"]}, Val Accuracy: {logs["val_sparse_categorical_accuracy"]}'),
    on_batch_end=lambda batch, logs: tqdm.update(1)
)

class ProgressBarCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epochs += 1
        self.pbar.update(1)
        self.pbar.set_postfix(logs, refresh=True)

    def on_train_begin(self, logs=None):
        self.epochs = 0
        self.pbar = tqdm(total=self.params['epochs'], unit='epoch', position=0)

    def on_train_end(self, logs=None):
        self.pbar.close()

In [113]:
print('Start training')
progress_bar_callback = ProgressBarCallback()
history = custom_model.fit(
    limited_train_dataset.shuffle(10).batch(batch_size).prefetch(tf.data.AUTOTUNE),
    epochs=num_epochs,
    validation_data=val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE),
    verbose=2,
    callbacks=[checkpoint_callback, progress_bar_callback]
)

Start training


  0%|          | 0/1 [39:47<?, ?epoch/s]



Epoch 1: val_loss improved from inf to 2.15234, saving model to /content/sample_data/model_checkpoint


100%|██████████| 1/1 [08:29<00:00, 509.38s/epoch, loss=1.98, sparse_categorical_accuracy=0, val_loss=2.15, val_sparse_categorical_accuracy=0]

1/1 - 509s - loss: 1.9844 - sparse_categorical_accuracy: 0.0000e+00 - val_loss: 2.1523 - val_sparse_categorical_accuracy: 0.0000e+00 - 509s/epoch - 509s/step


100%|██████████| 1/1 [08:29<00:00, 509.40s/epoch, loss=1.98, sparse_categorical_accuracy=0, val_loss=2.15, val_sparse_categorical_accuracy=0]


In [1]:
# Save the trained model if needed
custom_model.save(os.path.join(BASE_DIR, 'trained_model'))

NameError: ignored

In [ ]:
from keras.utils import Progbar

# Tokenize and preprocess the test data
test_encoded_statement_data = tokenizer(
    test['statement'].tolist(),
    padding=True,
    truncation=True,
    return_tensors='tf'
)
test['label_encoded'] = train['label'].map(label_mapping)

# Create TensorFlow dataset for testing
test_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': test_encoded_statement_data['input_ids'],
        'attention_mask': test_encoded_statement_data['attention_mask']
    },
    test['label_encoded']
))
limited_test_dataset = test_dataset.take(limit)
test_steps = len(limited_test_dataset)
progbar = Progbar(test_steps)

# Evaluate the model on the test dataset
results = custom_model.evaluate(limited_test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE))

# Display the results and update the progress bar
for metric_name, result in zip(custom_model.metrics_names, results):
    print(f'{metric_name}: {result}')

    # Update progress bar
    progbar.update(1)
# Print the evaluation results (including accuracy)
print("Test Loss:", results[0])
print("Test Accuracy:", results[1])